In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)
import os
import sys
import urllib.request
from urllib import parse
import ssl
from selenium.webdriver.support.ui import Select

In [16]:
driver = webdriver.Chrome('./driver/chromedriver.exe')

C:\Users\jkk18\AppData\Local\Temp\ipykernel_10764\3506397282.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver.exe')


In [17]:
base_url = 'https://www.wine21.com/13_search/wine_list.html'

In [18]:
driver.get(base_url)

In [19]:
# #shOrder1 요소 클릭
driver.find_element(By.CSS_SELECTOR, '#shOrder1').click()

# #shOrder1 > option:nth-child(4) 선택
select = Select(driver.find_element(By.CSS_SELECTOR, '#shOrder1'))
select.select_by_index(3)

In [20]:
import time

# 스크롤을 내릴 때마다 새로운 와인이 로드될 때까지 반복
for _ in range(40):
    # 현재 스크롤 위치 저장
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # 스크롤 내린 후 로딩 대기 시간
    time.sleep(3)

    # 더 보기 버튼 확인
    wine_list_more_btn = driver.find_element(By.CSS_SELECTOR, '#wineListMoreBtn')
    if wine_list_more_btn.is_displayed():
        # 더 보기 버튼 클릭
        wine_list_more_btn.click()
    else:
        # 더 이상 와인이 로드되지 않으면 반복 종료
        break

In [23]:
result = pd.DataFrame(columns=['당도', '바디', '산도', '타닌'])

i = 1
while True:
    sel_wine = '#wine_list > li:nth-child(' + str(i) + ') > div.txt-area > div.cnt-header > h3 > a'
    elem_wine = driver.find_element(By.CSS_SELECTOR, sel_wine)
    
    # 와인 클릭
    elem_wine.click()
    time.sleep(3)

    # 작업 수행
    new_frame = pd.DataFrame(columns=['당도', '바디', '산도', '타닌'])
    for a in range(1, 5):
        elems = driver.find_elements(By.CSS_SELECTOR, 'body > section > div.inner > div.clear > div.wine-top-right > div.wine-components > ul > li:nth-child(' + str(a) + ') > div > a')
        a_on_count = 0

        for elem in elems:
            if elem.tag_name == 'a' and 'on' in elem.get_attribute('class'):
                a_on_count += 1

        # 해당 열에 값을 추가
        if a == 1:
            new_frame.loc[0, '당도'] = a_on_count
        elif a == 2:
            new_frame.loc[0, '바디'] = a_on_count
        elif a == 3:
            new_frame.loc[0, '산도'] = a_on_count
        elif a == 4:
            new_frame.loc[0, '타닌'] = a_on_count

    # 새로운 프레임을 결과에 추가
    result = pd.concat([result, new_frame], ignore_index=True)

    # 이전 페이지로 돌아가기
    driver.back()
    time.sleep(3)

    # 다음 와인을 클릭하기 위해 i 값을 증가시킴
    i += 1

    # 더 이상 클릭할 와인이 없으면 반복 종료
    if not driver.find_elements(By.CSS_SELECTOR, sel_wine):
        break

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#wine_list > li:nth-child(411) > div.txt-area > div.cnt-header > h3 > a"}
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008AA813+48355]
	(No symbol) [0x0083C4B1]
	(No symbol) [0x00745358]
	(No symbol) [0x007709A5]
	(No symbol) [0x00770B3B]
	(No symbol) [0x0079E232]
	(No symbol) [0x0078A784]
	(No symbol) [0x0079C922]
	(No symbol) [0x0078A536]
	(No symbol) [0x007682DC]
	(No symbol) [0x007693DD]
	GetHandleVerifier [0x00B0AABD+2539405]
	GetHandleVerifier [0x00B4A78F+2800735]
	GetHandleVerifier [0x00B4456C+2775612]
	GetHandleVerifier [0x009351E0+616112]
	(No symbol) [0x00845F8C]
	(No symbol) [0x00842328]
	(No symbol) [0x0084240B]
	(No symbol) [0x00834FF7]
	BaseThreadInitThunk [0x773E7D59+25]
	RtlInitializeExceptionChain [0x7776B74B+107]
	RtlClearBits [0x7776B6CF+191]


In [24]:
result

,당도,바디,산도,타닌
0,1,3,3,3
1,1,3,3,2
2,2,4,3,1
3,1,4,3,3
4,4,2,4,3
...,...,...,...,...
405,1,3,4,4
406,1,4,4,4
407,1,3,3,1
408,1,4,4,5


In [25]:
result.to_csv('wine_character.csv',encoding='utf-8-sig')